In [1]:
import numpy as np

In [12]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced

midi_path = "pop1k7/midi_transcribed/src_001/3.midi"
MIDIPlayer(midi_path, 400)  
